In [1]:
import numpy as np
import sys
#np.set_printoptions(precision=5, linewidth=200, suppress=True)
import psi4
from helper_PFCI import PFHamiltonianGenerator
from helper_PFCI import Determinant
from helper_cqed_rhf import cqed_rhf
np.set_printoptions(threshold=sys.maxsize)

In [10]:
mol_str = """
0 1
O
H 1 1.0
H 1 1.0 2 104.0
symmetry c1
"""

options_dict = {'basis': 'sto-3g',
                  'scf_type': 'pk',
                  'e_convergence': 1e-10,
                  'd_convergence': 1e-10
                  }

mol = psi4.geometry(mol_str)

cavity_options = {
    'omega_value' : 0.36,
    'lambda_vector' : np.array([0, 0, 0.05]),
    'ci_level' : 'fci',
    'ignore_coupling' : False,
    'number_of_photons' : 1,
    'natural_orbitals' : False,
    'davidson_roots' : 4,
    'nact_orbs' : 0,
    'nact_els' : 0
}


psi4.set_options(options_dict)
psi4.core.set_output_file('output.dat', False)

In [11]:

# Build PF Hamiltonian matrix 
H2_PF = PFHamiltonianGenerator(mol_str, options_dict, cavity_options)



Start SCF iterations:

Canonical RHF One-electron energy = -121.6753455400170054
CQED-RHF One-electron energy      = -121.6676660938672399
Nuclear repulsion energy          = 8.8026031303907679
Dipole energy                     = 0.0001684105642096
SCF Iteration   1: Energy = -74.9602756380696320   dE = -7.49603E+01   dRMS = 1.50562E-04
SCF Iteration   2: Energy = -74.9602768487002038   dE = -1.21063E-06   dRMS = 5.08311E-05
SCF Iteration   3: Energy = -74.9602769853252084   dE = -1.36625E-07   dRMS = 1.94293E-05
SCF Iteration   4: Energy = -74.9602770091915716   dE = -2.38664E-08   dRMS = 8.48057E-06
SCF Iteration   5: Energy = -74.9602770137898915   dE = -4.59832E-09   dRMS = 3.74810E-06
SCF Iteration   6: Energy = -74.9602770147002389   dE = -9.10347E-10   dRMS = 1.67262E-06
SCF Iteration   7: Energy = -74.9602770148818820   dE = -1.81643E-10   dRMS = 7.47728E-07
SCF Iteration   8: Energy = -74.9602770149182192   dE = -3.63372E-11   dRMS = 3.34525E-07
SCF Iteration   9: Energy = -7

In [46]:
#print(H2_PF.CISdets[0]) #.getUniqueOrbitalsInListsPlusSign(H2_PF.CISdets[0]))
#for i in range(20):
#    print(i,H2_PF.CISdets[i])
_bra = H2_PF.CISdets[11].copy()
_ket = H2_PF.CISdets[1].copy()

_bra.getUniqueOrbitalsInListsPlusSign
print(_bra)
print(_ket)
#_ket.removeBetaOrbital(1)
#_bra.removeAlphaOrbital(1)
#print(_ket)
#print(_bra)
#u1, u2, sign = _bra.getUniqueOrbitalsInListsPlusSign(_ket)
#print(sign)

|[0, 1, 2, 3, 5][0, 1, 2, 3, 4]>
|[0, 1, 2, 3, 4][0, 1, 2, 3, 5]>


In [47]:
def computeSplusSminusPQ(bra, ket, p, q):
    # act on ket with S-(q)
    # S-(q) kills q_alpha and creates q_beta
    a, b = ket.getOrbitalIndexLists()
    print(a, b)

    # check to see if q_alpha is unnoccupied
    if (q not in a):
        return 0
    else:
        ket.removeAlphaOrbital(q)
    if (q not in b):
        ket.addBetaOrbital(q)
    else:
        return 0
    
    # act on updated ket with S+(p)
    # S+(p) kills p_beta and creates p_alpha
    
    # get updated occupation list
    a, b = ket.getOrbitalIndexLists()
    print(a, b)
    if (p not in b):
        return 0
    else:
        ket.removeBetaOrbital(p)
    if (p not in a):
        ket.addAlphaOrbital(p)
    else:
        return 0
    
    numUniqueOrbitals = bra.numberOfTotalDiffOrbitals(ket)
    print("numUnique",numUniqueOrbitals)
    if numUniqueOrbitals==0:
        u1, u2, sign = bra.getUniqueOrbitalsInListsPlusSign(ket)
        print(u1, u2, sign)
        return sign
    else:
        return 0
    

    

In [45]:
print(computeSplusSminusPQ(_bra, _ket, 5, 4))


[0, 1, 2, 3, 4] [0, 1, 2, 3, 5]
[0, 1, 2, 3] [0, 1, 2, 3, 4, 5]
0.0
([], []) ([], []) 1
1


In [ ]:
_H_spin = np.einsum("uj,vi,uv", H2_PF.Ca, H2_PF.Ca, H2_PF.H_1e_ao)
print(_H_spin)

In [ ]:
_H_spin = np.repeat(_H_spin, 2, axis=0)
print(_H_spin)

In [ ]:
_H_spin = np.repeat(_H_spin, 2, axis=1)
print(_H_spin)

In [ ]:
spin_ind = np.arange(_H_spin.shape[0], dtype=int) % 2
print(spin_ind)

In [ ]:
Hspin = _H_spin * (spin_ind.reshape(-1, 1) == spin_ind)
print(spin_ind.reshape(-1, 1) == spin_ind)
print(Hspin)

In [ ]:
        if self.ignore_coupling == False:
            self.H_1e_ao += self.q_PF_ao + self.d_PF_ao
        # build H_spin
        # spatial part of 1-e integrals
        _H_spin = np.einsum("uj,vi,uv", self.Ca, self.Ca, self.H_1e_ao)
        _H_spin = np.repeat(_H_spin, 2, axis=0)
        _H_spin = np.repeat(_H_spin, 2, axis=0)
        # spin part of 1-e integrals
        spin_ind = np.arange(_H_spin.shape[0], dtype=int) % 2
        # product of spatial and spin parts
        self.Hspin = _H_spin * (spin_ind.reshape(-1, 1) == spin_ind)

In [ ]:
# diagonalize the Hamiltonian matrix
e_fci, wavefunctions = np.linalg.eigh(H2_PF.H_PF)

# davidson eigenvalues and eigenvectors are stored in attributes
# H2_PF.cis_e and H2_PF.cis_c for now!


In [ ]:
print("checking to see if the Davidson Roots Agree!")
print(e_fci[:14])
print(H2_PF.cis_e[:14] )
print(np.allclose(e_fci[:14] , H2_PF.cis_e[:14]))


In [ ]:

H2_PF.calc1RDMfromCIS(wavefunctions[:,14])
print(np.trace(H2_PF.Dij))
print(np.trace(H2_PF.Dab))
print(np.trace(H2_PF.D1))



e1_test_rdm = np.einsum("pq,pq->", H2_PF.Hspin, H2_PF.D1)

temp=np.einsum("pq,q->p", H2_PF.H_1E,wavefunctions[:,14])
e1_test_wfn = np.dot(wavefunctions[:,14].T,temp)




In [ ]:
print(e1_test_rdm)
print(e1_test_wfn-H2_PF.Enuc)
print(e1_test_rdm-e1_test_wfn+H2_PF.Enuc)